# Build Docker Image

In this notebook, we build a docker image that we will later use with Azure ML service as custom docker image. We will push the image to Azure Container Registry.

In [1]:
from dotenv import set_key, find_dotenv

In [2]:
env_path = find_dotenv(raise_error_if_not_found=True)

In [3]:
image_name = 'YOUR_IMAGE_NAME' # e.g. torchdet

image_name = 'torchdet'

In [4]:
set_key(env_path, "image_name", image_name)

(True, 'image_name', 'torchdet')

In [5]:
%%writefile dockerfile

FROM nvidia/cuda:9.0-cudnn7-devel-ubuntu16.04

RUN apt-get update && apt-get install -y --no-install-recommends \
        build-essential \
        cmake \
        git \
        curl \
        ca-certificates && \
    rm -rf /var/lib/apt/lists/*

ENV PYTHON_VERSION=3.7
RUN curl -o ~/miniconda.sh -O  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh  && \
    chmod +x ~/miniconda.sh && \
    ~/miniconda.sh -b -p /opt/conda && \
    rm ~/miniconda.sh && \
    /opt/conda/bin/conda create -y --name py$PYTHON_VERSION python=$PYTHON_VERSION numpy scipy scikit-learn && \
    /opt/conda/bin/conda clean -ya && \
    ln -s /opt/conda/etc/profile.d/conda.sh /etc/profile.d/conda.sh && \
    echo ". /opt/conda/etc/profile.d/conda.sh" >> ~/.bashrc && \ 
    echo "conda activate py$PYTHON_VERSION" >> ~/.bashrc

ENV PATH /opt/conda/envs/py$PYTHON_VERSION/bin:$PATH
ENV LD_LIBRARY_PATH /opt/conda/envs/py$PYTHON_VERSION/lib:/usr/local/cuda/lib64/:$LD_LIBRARY_PATH
        
RUN /opt/conda/bin/conda install --name py$PYTHON_VERSION -c pytorch pytorch=1.1.0 torchvision=0.3.0 cudatoolkit=9.0 \
    matplotlib=3.1.0 Cython=0.29.10
    
RUN pip install --upgrade pip && \
    pip install --upgrade azureml-sdk[notebooks]==1.0.41 && \
    /opt/conda/bin/conda clean -yt

RUN git clone https://github.com/cocodataset/cocoapi.git && \
    cd cocoapi/PythonAPI && make && cd -

Overwriting dockerfile


In [11]:
acr_resource_group = "YOUR_ACR_RESOURCE_GROUP"
acr_location = "YOUR_ACR_LOCATION"
acr_name = "YOUR_ACR_NAME"

acr_resource_group = "fboyluacrrg"
acr_location = "eastus"
acr_name = "torchvisionacr"

In [20]:
set_key(env_path, "acr_resource_group", acr_resource_group)

(True, 'acr_resource_group', 'fboyluacrrg')

In [27]:
!az group create --name {acr_resource_group} --location {acr_location}

{
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourceGroups/fboyluacrrg",
  "location": "eastus",
  "managedBy": null,
  "name": "fboyluacrrg",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null
}


In [29]:
!az acr create --resource-group {acr_resource_group} --name {acr_name} --sku Basic

{
  "adminUserEnabled": false,
  "creationDate": "2019-06-27T16:15:23.482357+00:00",
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourceGroups/fboyluacrrg/providers/Microsoft.ContainerRegistry/registries/torchvisionacr",
  "location": "eastus",
  "loginServer": "torchvisionacr.azurecr.io",
  "name": "torchvisionacr",
  "provisioningState": "Succeeded",
  "resourceGroup": "fboyluacrrg",
  "sku": {
    "name": "Basic",
    "tier": "Basic"
  },
  "status": null,
  "storageAccount": null,
  "tags": {},
  "type": "Microsoft.ContainerRegistry/registries"
}


In [12]:
acr_server_name = '{}.azurecr.io'.format(acr_name )
acr_server_name

'torchvisionacr.azurecr.io'

In [13]:
set_key(env_path, "acr_server_name", acr_server_name)

(True, 'acr_server_name', 'torchvisionacr.azurecr.io')

In [35]:
!az acr login --name {acr_name}

Login Succeeded
WARNING! Your password will be stored unencrypted in /home/fboylu/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [46]:
!az acr update -n {acr_name} --admin-enabled true

{
  "adminUserEnabled": true,
  "creationDate": "2019-06-27T16:15:23.482357+00:00",
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourceGroups/fboyluacrrg/providers/Microsoft.ContainerRegistry/registries/torchvisionacr",
  "location": "eastus",
  "loginServer": "torchvisionacr.azurecr.io",
  "name": "torchvisionacr",
  "provisioningState": "Succeeded",
  "resourceGroup": "fboyluacrrg",
  "sku": {
    "name": "Basic",
    "tier": "Basic"
  },
  "status": null,
  "storageAccount": null,
  "tags": {},
  "type": "Microsoft.ContainerRegistry/registries"
}


In [42]:
image_tag = acr_server_name + '/' + image_name
image_tag

'torchvisionacr.azurecr.io/torchdet'

In [45]:
!docker build -t {image_tag} .

Sending build context to Docker daemon  523.9MB
Step 1/9 : FROM nvidia/cuda:9.0-cudnn7-devel-ubuntu16.04
 ---> 65dee97b9662
Step 2/9 : RUN apt-get update && apt-get install -y --no-install-recommends         build-essential         cmake         git         curl         ca-certificates &&     rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 2515c1b02bdd
Step 3/9 : ENV PYTHON_VERSION=3.7
 ---> Using cache
 ---> 5cc9f24e56de
Step 4/9 : RUN curl -o ~/miniconda.sh -O  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh  &&     chmod +x ~/miniconda.sh &&     ~/miniconda.sh -b -p /opt/conda &&     rm ~/miniconda.sh &&     /opt/conda/bin/conda create -y --name py$PYTHON_VERSION python=$PYTHON_VERSION numpy scipy scikit-learn &&     /opt/conda/bin/conda clean -ya &&     ln -s /opt/conda/etc/profile.d/conda.sh /etc/profile.d/conda.sh &&     echo ". /opt/conda/etc/profile.d/conda.sh" >> ~/.bashrc &&     echo "conda activate py$PYTHON_VERSION" >> ~/.bashrc
 ---> Using cache
 

In [47]:
!docker push {image_tag}

The push refers to repository [torchvisionacr.azurecr.io/torchdet]

82a0dce1: Preparing 
aec1a673: Preparing 
46f5794a: Preparing 
009b80ee: Preparing 
68871e46: Preparing 
eb3abf5c: Preparing 
7198b4a5: Preparing 
b1166576: Preparing 
61ff09c5: Preparing 
208b2662: Preparing 
a9223884: Preparing 
072a5034: Preparing 
2298c779: Preparing 
78fcccca: Preparing 


09b80ee: Pushing  767.6MB/1.287GBPushing  184.7MB/346.3MBPushing   2.77MB/767MBPushing  669.6MB/1.287GB

6f5794a: Pushing   2.19GB/2.975GBPushing  1.037GB/1.287GBPushing  1.191GB/1.287GBPushing  1.735GB/2.975GB

6f5794a: Pushed   3.003GB/2.975GBlatest: digest: sha256:763ab1043957413495b0fd25eba9affdd59b8d6bee4eb83ffd122d941bf2a9bc size: 3482


In [14]:
acr_username = !az acr credential show -n {acr_name} --query username

In [15]:
acr_username = acr_username[0][1:-1]
acr_username

'torchvisionacr'

In [16]:
set_key(env_path, "acr_username", acr_username)

(True, 'acr_username', 'torchvisionacr')

In [17]:
acr_password = !az acr credential show -n {acr_name} --query passwords[0].value

In [18]:
acr_password = acr_password[0][1:-1]
acr_password

'0VrrxpuofLdA32=fhdddgIrO7nUm8Z6y'

In [19]:
set_key(env_path, "acr_password", acr_password)

(True, 'acr_password', '0VrrxpuofLdA32=fhdddgIrO7nUm8Z6y')

You can now move on to the next notebook to tune the hyperparameters of the object detection model.